In [38]:
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [39]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=0.1)

        position = torch.arange(0, max_seq_length).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-torch.log(torch.tensor(10000.0)) / d_model))
        pos_encoding = torch.zeros(max_seq_length, d_model)
        pos_encoding[:, 0::2] = torch.sin(position * div_term)
        pos_encoding[:, 1::2] = torch.cos(position * div_term)
        pos_encoding = pos_encoding.unsqueeze(0)

        self.register_buffer('pos_encoding', pos_encoding)

    def forward(self, x):
        x = x + self.pos_encoding[:, :x.size(1)].clone().detach()
        return self.dropout(x)

In [40]:
class ByteFormer(nn.Module):
    def __init__(self,d_model,nhead,num_layers,embedding_dim,kernel_size,stride,num_classes,max_seq_length):
        super(ByteFormer, self).__init__()
        self.embedding_layer = nn.Embedding(256,embedding_dim)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)
        self.conv = nn.Conv1d(embedding_dim,embedding_dim,kernel_size,stride)
        self.encoder_layers = nn.ModuleList([
            nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead)
            for _ in range(num_layers)
        ])
        self.classifier = nn.Linear(embedding_dim,num_classes)
        self.output = nn.Softmax(dim=0)
    def forward(self,x):
        x = self.embedding_layer(x)
        x = self.positional_encoding(x)
        x = torch.transpose(x,1,2)
        x = torch.transpose(self.conv(x),2,1)
        for encoder_layer in self.encoder_layers:
            x = encoder_layer(x)
        n = x.shape[1]
        x = torch.sum(x,dim=1)/n
        x = self.classifier(x)
        x = self.output(x)
        return x

In [41]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [42]:
path1 = '/home/notebook/data/group/cifar-10-batches-py/data_batch_1'
path2 = '/home/notebook/data/group/cifar-10-batches-py/data_batch_2'
path3 = '/home/notebook/data/group/cifar-10-batches-py/data_batch_3'
path4 = '/home/notebook/data/group/cifar-10-batches-py/data_batch_4'
path5 = '/home/notebook/data/group/cifar-10-batches-py/data_batch_5'
path_test = '/home/notebook/data/group/cifar-10-batches-py/test_batch'
path_d = '/home/notebook/data/group/cifar-10-batches-py/batches.meta'

In [43]:
data_batch_1 = unpickle(path1)
data_batch_2 = unpickle(path2)
data_batch_3 = unpickle(path3)
data_batch_4 = unpickle(path4)
data_batch_5 = unpickle(path5)
test_batch = unpickle(path_test)
batches_meta = unpickle(path_d)

In [44]:
data_batch_1.keys()

dict_keys([b'batch_label', b'labels', b'data', b'filenames'])

In [45]:
test_batch.keys()

dict_keys([b'batch_label', b'labels', b'data', b'filenames'])

In [46]:
batches_meta.keys()

dict_keys([b'num_cases_per_batch', b'label_names', b'num_vis'])

In [47]:
training_data1,training_labels1 = torch.tensor(data_batch_1[b'data']).to(torch.int),data_batch_1[b'labels']
training_data2,training_labels2 = torch.tensor(data_batch_2[b'data']).to(torch.int),data_batch_2[b'labels']
training_data3,training_labels3 = torch.tensor(data_batch_3[b'data']).to(torch.int),data_batch_3[b'labels']
training_data4,training_labels4 = torch.tensor(data_batch_4[b'data']).to(torch.int),data_batch_4[b'labels']
training_data5,training_labels5 = torch.tensor(data_batch_5[b'data']).to(torch.int),data_batch_5[b'labels']
test_data,test_labels = torch.tensor(test_batch[b'data']).to(torch.int),test_batch[b'labels']

In [49]:
# print(test_labels.shape)

In [50]:
# Set random seed for reproducibility
torch.manual_seed(42)

# Define hyperparameters
batch_size = 64
learning_rate = 0.00008
num_epochs = 5

# Convert training and test data to tensors
k = 100
training_data = training_data1[:k]
test_data = test_data[:k]
training_labels = torch.LongTensor(training_labels1[:k])
test_labels = torch.LongTensor(test_labels[:k])

train_dataset = TensorDataset(training_data.to(device), training_labels.to(device))
test_dataset = TensorDataset(test_data.to(device), test_labels.to(device))



model = ByteFormer(192,3,9,192,32,16,10,3072)
# Define the loss function and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0

    # Mini-batch training loop
    for inputs,labels in train_loader:

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Track training loss and accuracy
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Calculate average training loss and accuracy
    train_loss /= (len(train_loader))
    train_accuracy = 100 * correct / total

    # Evaluate the model on the test set
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        # Evaluation on the test set
        for inputs,labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate average test loss and accuracy
    test_loss /= (len(test_loader))
    test_accuracy = 100 * correct / total

    # Print training progress
    print(f"Epoch [{epoch+1}/{num_epochs}] - Train Loss: {train_loss:.4f} - Train Accuracy: {train_accuracy:.2f}% - "
          f"Test Loss: {test_loss:.4f} - Test Accuracy: {test_accuracy:.2f}%")

# Save the trained model
# torch.save(model.state_dict(), 'model.pth')


3072
100
Epoch [1/5] - Train Loss: 2.3025 - Train Accuracy: 5.00% - Test Loss: 2.3024 - Test Accuracy: 11.00%
Epoch [2/5] - Train Loss: 2.3012 - Train Accuracy: 52.00% - Test Loss: 2.3019 - Test Accuracy: 12.00%
Epoch [3/5] - Train Loss: 2.2994 - Train Accuracy: 71.00% - Test Loss: 2.3008 - Test Accuracy: 13.00%
Epoch [4/5] - Train Loss: 2.2961 - Train Accuracy: 84.00% - Test Loss: 2.2973 - Test Accuracy: 16.00%
Epoch [5/5] - Train Loss: 2.2953 - Train Accuracy: 78.00% - Test Loss: 2.2924 - Test Accuracy: 14.00%


In [ ]:
import os
files = os.listdir("./mypart2")
files2 = os.listdir("./gt2")
for file in files:
    if file not in files2:
        print(file)